<a href="https://colab.research.google.com/github/FerDoranNie/rag_consar_nerdfore/blob/master/Test/AFORE_rag_consar_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --quiet playwright
!playwright install

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 MB 55.7 MB/s eta 0:00:00
173.9 MiB [] 0% 0.0s173.9 MiB [] 0% 3.1s173.9 MiB [] 0% 6.1s173.9 MiB [] 1% 3.5s173.9 MiB [] 2% 4.3s173.9 MiB [] 2% 4.2s173.9 MiB [] 3% 3.9s173.9 MiB [] 4% 3.3s173.9 MiB [] 5% 3.2s173.9 MiB [] 5% 3.4s173.9 MiB [] 6% 3.4s173.9 MiB [] 6% 3.2s173.9 MiB [] 7% 3.1s173.9 MiB [] 8% 3.0s173.9 MiB [] 9% 2.9s173.9 MiB [] 9% 3.2s173.9 MiB [] 10% 3.2s173.9 MiB [] 10% 3.4s173.9 MiB [] 11% 3.2s173.9 MiB [] 12% 3.0s173.9 MiB [] 13% 3.0s173.9 MiB [] 14% 2.8s173.9 MiB [] 15% 2.8s173.9 MiB [] 16% 2.8s173.9 MiB [] 17% 2.8s173.9 MiB [] 18% 2.7s173.9 MiB [] 19% 2.6s173.9 MiB [] 20% 2.4s173.9 MiB [] 21% 2.3s173.9 MiB [] 23% 2.3s173.9 MiB [] 24% 2.2s173.9 MiB [] 25% 2.2s173.9 MiB [] 26% 2.1s173.9 MiB [] 27% 2.1s173.9 MiB [] 28% 2.0s173.9 MiB [] 28% 2.1s173.9 MiB [] 29% 2.0s173.9 MiB [] 31% 2.0s173.9 MiB [] 32% 1.9s173.9 MiB [] 33% 1.8s173.9 MiB [] 34% 1.8s173.9 MiB [] 35% 1.7s173.9 MiB [] 36% 1.7s173.9 MiB [] 37% 1.6s173.9 MiB [] 

In [ ]:
!pip install --quiet docling langextract langchain openai

## Script para AFORE RAG

Vamos a crear el script para el RAG de los datos de AFORE de la CONSAR

Pasos

1. Generar script para descarga automatizada    
2. Detectar si los archivos generados ya fueron descargados, de lo contrario no descargarlos
3. Transformar los datos a texto simple usando Docling



In [ ]:
import os
import sys
import asyncio ## Los métodos asíncronos
import pandas as pd
import os
from playwright.async_api import async_playwright
from datetime import datetime
import shutil
import time
import langextract as lx
from google import genai

from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
import re
import unicodedata


In [ ]:
from google.colab import ai

ai.list_models()

In [ ]:
### Importando secretos
from google.colab import userdata
### Generando datos desde Colab
drive_api_key = userdata.get('DRIVE_API_KEY')
gemini_api_key = userdata.get('Gemini_secret')
openai_api_key = userdata.get('openai_api_key')
## Cambiar esto después a os para los secretos en GitHub Actions

In [ ]:
import asyncio
from playwright.async_api import async_playwright
async def main():
   async with async_playwright() as p:
       print('Comienza test')
       browser = await p.chromium.launch()
       page = await browser.new_page()
       await page.goto("https://www.google.com/")
       print(await page.title())
       #await browser.close()



In [ ]:
drive_folder_path = '/content/drive/MyDrive/RAG_CONSAR'


In [ ]:
## Función para remover acentos

def remove_accents(input_str):
    if not isinstance(input_str, str):
        raise TypeError("Input must be a string.")

    # Normalize to NFD form (decomposes characters into base + accents)
    normalized = unicodedata.normalize('NFD', input_str)

    # Filter out combining characters (accents)
    without_accents = ''.join(
        char for char in normalized
        if unicodedata.category(char) != 'Mn'
    )

    return without_accents



In [ ]:
test_strings = ['ratón', 'ídolo', 'ñame', 'újule', 'época', 'patán' ]
for cadena in test_strings:
  print(remove_accents(cadena))

raton
idolo
name
ujule
epoca
patan


In [ ]:
def clean_string(string_to_clean, with_underscore=False):
  string_cleaned = re.sub(r'_', ' ', string_to_clean)
  ## Cambiar acentos por letras sin acentos, también aplica la ñ
  string_cleaned = remove_accents(string_cleaned)
  ## Remover caracteres raros
  string_cleaned = re.sub(r'[^\w\s]', '', string_cleaned)
  ## Convert to UTF8
  string_cleaned = string_cleaned.encode('utf-8').decode('utf-8')
  string_cleaned = re.sub(r'\s+', ' ', string_cleaned)
  ### Remover espacios al inicio y al final
  string_cleaned = string_cleaned.strip()
  if with_underscore:
    return  string_cleaned.replace(' ', '_').lower()
  else:
    return string_cleaned.lower()

In [ ]:
test = "Me decían que estaba loco {} por donde fuera siempre veía un ñandu, ###"
clean_string(test, with_underscore=False)

'me decian que estaba loco por donde fuera siempre veia un nandu'

In [ ]:
async def descargar_pdfs_consar(dir_to_download_files, existing_df=None):
    #download_dir = "./descargas_consar"
    download_dir = dir_to_download_files
    if not os.path.exists(download_dir):
        os.makedirs(download_dir)

    # Prepare a set of already downloaded file identifiers for quick lookup
    # We will use the generated filename as the identifier, e.g., "NOMBRE_0.pdf"
    ### Verificar que el nombre de la path exista
    downloaded_file_names = set()
    if existing_df is not None and not existing_df.empty:
        downloaded_file_names = set(existing_df['archivo_descargado'].dropna().tolist())

    datos = []

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        context = await browser.new_context(
            accept_downloads=True,
            viewport={'width': 1920, 'height': 1080}
        )
        page = await context.new_page()
        print("Navegando a la página...")
        await page.goto('https://www.consar.gob.mx/gobmx/aplicativo/SINORconsar#')
        await page.wait_for_selector('table', timeout=30000)
        print("Tabla detectada, extrayendo datos...")

        filas = await page.query_selector_all('table tbody tr')

        for i, fila in enumerate(filas):
            try:
                celdas = await fila.query_selector_all('td')
                if len(celdas) >= 3:
                    nombre = (await celdas[0].inner_text()).strip()
                    print('Nombre')
                    tipo = (await celdas[1].inner_text()).strip()
                    fecha_dof = (await celdas[2].inner_text()).strip()

                    expected_filename = f"{nombre}_{i}.pdf"
                    ruta_completa = os.path.join(download_dir, expected_filename)

                    # Check if already downloaded and file exists
                    if expected_filename in downloaded_file_names and os.path.exists(ruta_completa):
                        print(f"Saltando: '{nombre}' (ya descargado como '{expected_filename}')")
                        datos.append({
                            'nombre': nombre,
                            'tipo': tipo,
                            'fecha_publicacion_dof': fecha_dof,
                            'archivo_descargado': expected_filename,
                            'ruta_completa': ruta_completa
                        })
                        continue # Skip to next row

                    print(f"Procesando: {nombre}")
                    archivo_descargado = None
                    try:
                        enlace_nombre = await celdas[0].query_selector('a[href*=".pdf"], a[href*="download"]')
                        if enlace_nombre:
                            async with page.expect_download(timeout=60000) as download_info:
                                await enlace_nombre.click()
                                await page.wait_for_timeout(500)
                            download = await download_info.value
                            if download.suggested_filename.lower().endswith('.pdf'):
                                archivo_descargado = expected_filename
                                await download.save_as(ruta_completa)
                                print(f"  ✓ Descargado desde nombre: {archivo_descargado}")

                    except Exception as e:
                        print(f"  ✗ Error descargando desde nombre para '{nombre}': {e}")

                    if not archivo_descargado:
                        try:
                            # Try to find a download button/link more generically in the row
                            boton_descargar = await fila.query_selector('button:has-text("Descargar"), a:has-text("Descargar"), a[href*=".pdf"], a[href*="download"]')
                            if boton_descargar:
                                async with page.expect_download(timeout=60000) as download_info: # Increased timeout
                                    await boton_descargar.click()
                                    await page.wait_for_timeout(500) # Small wait
                                download = await download_info.value
                                if download.suggested_filename.lower().endswith('.pdf'):
                                    archivo_descargado = expected_filename
                                    await download.save_as(ruta_completa)
                                    print(f"  ✓ Descargado desde botón: {archivo_descargado}")

                        except Exception as e:
                            print(f"  ✗ Error descargando desde botón para '{nombre}': {e}")

                    datos.append({
                        'nombre': nombre,
                        'tipo': tipo,
                        'fecha_publicacion_dof': fecha_dof,
                        'archivo_descargado': archivo_descargado,
                        'ruta_completa': ruta_completa if archivo_descargado else None
                    })

            except Exception as e:
                print(f"Error procesando fila {i} ('{nombre if 'nombre' in locals() else 'N/A'}'): {e}")
                continue

        await browser.close()
    return datos, download_dir

def crear_dataframe_y_guardar(datos, download_dir):
    # Crear DataFrame
    df = pd.DataFrame(datos)

    # Guardar DataFrame como CSV
    csv_path = os.path.join(download_dir, "metadata_documentos.csv")
    df.to_csv(csv_path, index=False, encoding='utf-8')

    print(f"\n✅ Proceso completado:")
    print(f"   - Documentos procesados: {len(df)}")
    print(f"   - PDFs descargados: {df['archivo_descargado'].notna().sum()}")
    print(f"   - CSV guardado en: {csv_path}")
    print(f"   - PDFs guardados en: {download_dir}")

    return df

# Función para subir a Google Drive (opcional)
def subir_a_drive(local_folder, drive_folder_id=None):
    """
    Función para subir archivos a Google Drive
    Requiere: pip install google-api-python-client google-auth-httplib2 google-auth-oauthlib
    """
    try:

        # Configurar autenticación (necesitas configurar las credenciales OAuth 2.0)
        # service = build('drive', 'v3', credentials=creds)

        # Subir archivos
        for filename in os.listdir(local_folder):
            file_path = os.path.join(local_folder, filename)
            if os.path.isfile(file_path):
                file_metadata = {
                    'name': filename,
                    'parents': [drive_folder_id] if drive_folder_id else []
                }
                media = MediaFileUpload(file_path, resumable=True)
                # file = service.files().create(body=file_metadata, media_body=media, fields='id').execute()
                print(f"Subido a Drive: {filename}")

    except ImportError:
        print("Bibliotecas de Google Drive no instaladas. Instala con: pip install google-api-python-client google-auth-httplib2 google-auth-oauthlib")
    except Exception as e:
        print(f"Error subiendo a Drive: {e}")

In [ ]:
# Load existing metadata if available
download_dir = "./descargas_consar"
metadata_path = os.path.join(download_dir, "metadata_documentos.csv")
existing_df = None
if os.path.exists(metadata_path):
    print(f"Cargando metadatos existentes desde {metadata_path}...")
    existing_df = pd.read_csv(metadata_path)
    print(f"  {len(existing_df)} registros encontrados.")

# Execute the download function, passing existing metadata
print("Iniciando la descarga de documentos...")
datos, download_dir = await descargar_pdfs_consar(existing_df=existing_df)
df = crear_dataframe_y_guardar(datos, download_dir)

# Display summary
print("\n📊 Resumen del DataFrame:")
display(df.head())
print(f"\nColumnas: {df.columns.tolist()}")

Navegando a la página...
Tabla detectada, extrayendo datos...
Procesando: 
  ✗ Error descargando desde botón: Timeout 30000ms exceeded while waiting for event "download"
=========================== logs ===========================
waiting for event "download"
Procesando: 


CancelledError: 